In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import networkx
import Graph

g = Graph.buildGraph()
# mulGraph = Graph.buildGraph()

Making paths from route: 3, var 5

Making paths from route: 3, var 6

Making paths from route: 1, var 1

Making paths from route: 1, var 2

Making paths from route: 7, var 13

...

Making paths from route: 313, var 1

Making paths from route: 313, var 2

Making paths from route: 337, var 1

Making paths from route: 337, var 2

Number of subgraphs: 1

In [ ]:
from math import sqrt
from rtree import index
from rich import print

def connectGraph(subGraphs: list[networkx.MultiDiGraph]):
    connectionPair = []
    for i in range(len(subGraphs)):
        for j in range(i + 1, len(subGraphs)):
            idx = index.Index()
            
            nodeCoordI = networkx.get_node_attributes(subGraphs[i], 'coord')
            
            iNodes = list(subGraphs[i].nodes)
            jNodes = list(subGraphs[j].nodes)
            # print(len(list(subGraphs[i].nodes)))
            # print(subGraphs[i].nodes[7278]['coord'])
            # print(subGraphs[i].has_node(7276))
            
            for node in iNodes:
                idx.insert(node, (nodeCoordI[node][0], nodeCoordI[node][1], nodeCoordI[node][0], nodeCoordI[node][1]))
                
            # print(jNodes)
            minDisSoFar = -1
            connection1 = -1
            connection2 = -1
            for node in jNodes:
                # print(subGraphs[j].nodes[7529]['coord'])
                x, y = subGraphs[j].nodes[node]['coord']
                nearestId = list(idx.nearest((x, y, x, y), 1))[0]
                x1, y1 = nodeCoordI[nearestId]
                
                dis = sqrt((x - x1)**2 + (y - y1)**2)
                if minDisSoFar == -1 or minDisSoFar > dis:
                    minDisSoFar = dis
                    connection1 = node
                    connection2 = nearestId
                    
            
            if connection1 == 1:
                print("No points found")
                return
            
            # Only connect two stops that are less than 1km apart
            if minDisSoFar < 1000:
                connectionPair.append((connection1, connection2))
                
    return connectionPair
    
    
                
                
connectGraph(mulGraph)

In [ ]:
connectGraph(mulGraph)

In [ ]:
def createPath(graph: networkx.MultiDiGraph, stopIdNode: list[str], stopPoints: list[tuple], pathPoints: list[tuple], runningTime, routeVarId):
    pathIndex = -1
    totalLength = LineString(pathPoints).length
    for i in range(len(stopIdNode)):
        if pathIndex == -1:
            pathIndex += 1
            continue
        
        closestIndex = findNextClosest(pathIndex, stopPoints[i], pathPoints)
        line = LineString(pathPoints[pathIndex : closestIndex + 1])
        pathIndex = closestIndex
        # print(stopIdNode[i - 1], stopIdNode[i], line.length)
        graph.add_edge(stopIdNode[i - 1], stopIdNode[i], time = runningTime / totalLength * line.length, dis = line.length, routeVar = routeVarId)
        networkx.set_node_attributes(graph, {i - 1: {"coord": stopPoints[i - 1]}})
        networkx.set_node_attributes(graph, {i: {"coord": stopPoints[i]}})
        
    return graph

In [ ]:
from rich import print
timeLabel = networkx.get_edge_attributes(g, 'time')
print(g.nodes[35]['coord'])
print(g[7526][7529][0]['time'])

In [ ]:

from networkx import connected_components


ug = g.to_undirected()
k = g.subgraph(list(connected_components(ug))[3])

subGraphs = [g.subgraph(com) for com in list(component for component in list(connected_components(ug)))]
# print(subGraphs[0])
# [g.subgraph(component for component in list(connected_components(ug)) )]
# gs = [sub for sub in ]
# print(k)
# print(k.nodes)

# networkx.draw_spring(k, with_labels = True)



In [5]:
import heapq

# def Dijkstra1(g: networkx.MultiDiGraph, index = 0):

heap = [[2, 35]]
heapq.heappush(heap, [1, 3]) 
heapq.heappush(heap, [3, 3])

heap 

[[1, 3], [2, 35], [3, 3]]

In [12]:
from rich import print
def Dijkstra1(g: networkx.MultiDiGraph, index = 0):
    minHeap = [[0, list(g.nodes)[index], list(g.nodes)[index]]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in list(g.nodes):
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
            
    return shortestPath

shortest = Dijkstra1(g, 0)
print(shortest) 

{
    35: (35, 0),
    7276: (35, 0.721166702694151),
    1451: (35, 1.5459098069235921),
    86: (35, 1.8286638195142664),
    7277: (7276, 1.8328662072033386),
    27: (1451, 2.150786895917856),
    121: (1451, 2.18514899238307),
    89: (35, 2.2476929398233807),
    1344: (1451, 2.309250030631589),
    26: (7276, 2.3099690415023635),
    88: (86, 2.46936746173906),
    7278: (7277, 2.6059781197124288),
    90: (89, 2.9205943484734935),
    122: (121, 3.1172756787784652),
    385: (89, 3.2145483363235594),
    1019: (86, 3.2856531190504654),
    120: (7277, 3.295268602270847),
    28: (27, 3.51676034617598),
    3080: (120, 3.7094955881293945),
    384: (385, 3.8141258266482927),
    425: (90, 3.828997302327602),
    1409: (90, 3.911834788222743),
    2976: (122, 3.936145936035336),
    1256: (28, 3.948883464453774),
    387: (385, 4.110373147113838),
    424: (425, 4.220468761232841),
    123: (122, 4.291224381941213),
    7588: (1256, 4.333521404510069),
    4275: (7278, 4.377267499952517),
    7264: (27, 4.512631037367312),
    30: (27, 4.540403160446298),
    7265: (27, 4.5538324026023265),
    7266: (27, 4.625557867529458),
    32: (7588, 4.703348495400987),
    2973: (2976, 4.737668420323665),
    7274: (3080, 4.741359280706961),
    7268: (7278, 4.755027595726004),
    7267: (27, 4.792501385966197),
    7693: (7266, 4.822637773474051),
    3691: (1256, 4.827254939272841),
    84: (3080, 4.852494651673716),
    1413: (1409, 4.881078442187711),
    2114: (32, 4.9373316459530425),
    3164: (384, 5.021091253597538),
    1196: (1344, 5.068572775626188),
    7564: (4275, 5.090425313575943),
    386: (387, 5.105609250248354),
    1297: (35, 5.137213559142487),
    426: (424, 5.2219007857999715),
    2115: (2114, 5.281206516919634),
    2028: (3691, 5.303995148615376),
    124: (123, 5.306002341483181),
    2975: (2973, 5.406355414665827),
    4271: (1256, 5.47048722289634),
    85: (84, 5.598125287968502),
    853: (7564, 5.643545549443733),
    3182: (387, 5.69863240090516),
    78: (2028, 5.90335047182983),
    3478: (1297, 5.9171429414737355),
    1416: (1413, 5.918538780098295),
    7275: (7274, 5.948761029116973),
    7270: (2028, 5.9837770642441654),
    3011: (1196, 6.017209045614507),
    2854: (7564, 6.023800168624938),
    3171: (386, 6.028507853699463),
    1252: (1344, 6.032124127579214),
    292: (1297, 6.046656018424487),
    7269: (2028, 6.0637028430217415),
    1194: (1196, 6.0955741600464926),
    82: (2028, 6.105204093581378),
    380: (1297, 6.149147645764376),
    2724: (4271, 6.209225970445096),
    3076: (1019, 6.245144516611212),
    4223: (2028, 6.418626533479348),
    87: (85, 6.532331174373324),
    2116: (2115, 6.545152156103426),
    429: (426, 6.656846010749342),
    431: (3182, 6.712257417588227),
    1254: (1252, 6.82387656832012),
    1296: (1297, 6.920540980479446),
    1891: (1416, 6.942758561981835),
    3079: (3076, 6.9460597692429715),
    3014: (3011, 6.966332229064008),
    1909: (3171, 6.998381743395194),
    125: (124, 7.040244296920644),
    388: (386, 7.049468589306956),
    1411: (1416, 7.055298022902358),
    1198: (1194, 7.084727919052206),
    433: (3182, 7.185318096590895),
    1882: (3478, 7.206542897079639),
    4272: (2724, 7.211828999552728),
    242: (292, 7.340668067413331),
    7273: (78, 7.34123028709846),
    383: (380, 7.389180260521766),
    31: (32, 7.4124201414658994),
    1944: (7269, 7.428648189578832),
    2405: (3164, 7.631180702268994),
    3477: (1296, 7.74254815450366),
    855: (2116, 7.849655474644754),
    92: (1909, 7.870031530667568),
    427: (429, 7.9077396843263585),
    1021: (2975, 7.914159506119666),
    287: (3164, 7.919319897178192),
    4273: (4272, 7.9936936917443955),
    3013: (3014, 7.9975858233034005),
    390: (388, 8.014964112341323),
    1299: (1296, 8.104972318586743),
    3078: (3079, 8.189050138675247),
    1195: (853, 8.20646683838151),
    856: (853, 8.235809727553697),
    1889: (1882, 8.264538116710774),
    126: (125, 8.2678152

In [14]:
shortest[4086]

(7119, 54.758798937168926)

In [4]:
# Dijkstra
def Dijkstra(k: networkx.MultiDiGraph, startingIndex = 0):
    unvisited = list(k.nodes)
    visited = [unvisited.pop(startingIndex)]

    if len(list(networkx.neighbors(k, visited[-1]))) == 0:
        # print("No path to any other node found")
        return
        
    shortestPath = {}
    for node in list(k.nodes):
        if node == visited[0]:
            shortestPath[node] = (node, 0)
        else:
            shortestPath[node] = (-1, -1)
            
    # print(shortestPath)

    while len(unvisited) > 0:
        nodeToConsider = visited[-1]
        
                
        # Update shortestPath
        for node in unvisited:
            dis = shortestPath[node][1]
            
            # Skip node that cant be reached directly
            if networkx.is_path(k, [nodeToConsider, node]) == False:
                continue
            
            weighToHere = shortestPath[nodeToConsider][1]
            minWeight = k[nodeToConsider][node][0]['time']
            if dis == -1:
                shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
            elif shortestPath[node][1] > minWeight + weighToHere:
                    shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
        
        nextNodeToConsider = -1
        minWeightSoFar = -1
        for u in unvisited:
            # Skip node that are not updated yet
            if shortestPath[u][1] == -1:
                continue
            
            # print(shortestPath[u][1])
            if minWeightSoFar == -1:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
            elif shortestPath[u][1] < minWeightSoFar:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
        if nextNodeToConsider == -1:
            # print("Unreachable node found")
            return shortestPath
            
        
        # print(shortestPath)
        # print(f"{nextNodeToConsider=}")
        
        unvisited.remove(nextNodeToConsider)
        visited.append(nextNodeToConsider)
        # break

    # print()
    return shortestPath

Dijkstra(g)

{35: (35, 0),
 7276: (35, 0.721166702694151),
 7277: (7276, 1.8328662072033386),
 7278: (7277, 2.6059781197124288),
 7265: (27, 4.5538324026023265),
 7266: (27, 4.625557867529458),
 7693: (7266, 4.822637773474051),
 1256: (28, 3.948883464453774),
 7588: (1256, 4.333521404510069),
 32: (7588, 4.703348495400987),
 31: (32, 7.4124201414658994),
 34: (31, 8.43327155645181),
 42: (34, 9.577961000535128),
 44: (42, 10.607138172691117),
 39: (44, 11.060212993246752),
 41: (291, 10.95596227026169),
 43: (41, 12.795312175866634),
 46: (43, 13.851856770717633),
 47: (46, 14.607113204325941),
 45: (47, 16.503406183176466),
 49: (45, 17.36419426634077),
 48: (49, 18.74438542540705),
 50: (48, 19.488993222037102),
 51: (50, 20.508482042191012),
 52: (51, 21.6800526002089),
 53: (52, 22.405671107387523),
 54: (53, 23.647215418351482),
 55: (54, 24.475635754425888),
 56: (4749, 22.221860068180213),
 57: (56, 23.8493370298128),
 60: (57, 25.90131500157969),
 58: (60, 27.758853562885193),
 62: (58, 29.

In [ ]:
# altered: networkx.MultiDiGraph = g.subgraph(list(connected_components(ug))[3]).copy()
# altered.add_node(13)
# altered.add_node(15)
# altered.add_edge(12, 7486, time = 12)
for i in range(len(list(g.nodes))):
    print("Starting at:", list(g.nodes)[i])
    shortestPath = Dijkstra(g, i)

    print(shortestPath)

In [ ]:
from rich import print
# print(g.subgraph(list(connected_components(ug))[0]))
grph: networkx.MultiDiGraph = g.subgraph(list(connected_components(ug))[0]).copy()
stops = list(grph.nodes)
stops.sort()
print(len(stops))